In [1]:
import pandas as pd
from tqdm import tqdm

batchX = '/home/bupi/Documents/pdy/hs/hsle/data/exportcomments-outputs/20200413_20200420/processed/merged.csv'

In [2]:
df = pd.read_csv(batchX)
df['profile_id'] = [p.strip().split()[-1] for p in df['Profile ID']]
print(df.shape)
df.head(2)

(124934, 16)


,cId,nId,Name (click to view profile),Profile ID,Date,Likes,Comment,(view source),postId,nChars,MsgUni,atleast1MM,MsgUniSeg,LexFound,PostURL,profile_id
0,1,0,Kyaw Thet,ID: 100017924341255,2020-04-13 17:52:10,0,သင့္ေၾကာင့္ကိုဗစ္မကူးပါေစနဲ႔လို,view comment,20200413_0337,31,သင့်ကြောင့်ကိုဗစ်မကူးပါစေနဲ့လို,True,သင့် ကြောင့် ကို ဗစ် မ ကူး ပါစေ နဲ့ လို,NaN,https://www.facebook.com/themyanmartimes/posts...,100017924341255
1,2,0,Chitmoe Thwe,ID: 100009696576683,2020-04-13 17:54:15,0,၆၂ေယာက္ရွိသြားၿပီဗ်ာသတင္းကေနာက္က်တယ္,view comment,20200413_0337,36,၆၂ယောက်ရှိသွားပြီဗျာသတင်းကနောက်ကျတယ်,True,၆၂ ယောက် ရှိ သွား ပြီ ဗျာ သတင်း က နောက်ကျ တယ်,NaN,https://www.facebook.com/themyanmartimes/posts...,100009696576683


## Get Unique Accounts


In [3]:
accounts = df.profile_id.unique()
print(accounts.shape)

(95201,)


## Comments Counts by Accounts

In [4]:
cmt_qty_by_acc = df.groupby('profile_id')['profile_id'].count()
print(cmt_qty_by_acc.shape)
cmt_qty_by_acc.sort_values(ascending=False, inplace=True)
cmt_qty_by_acc[:2]

(95201,)


profile_id
100002366838046    40
100000478433815    37
Name: profile_id, dtype: int64

### Filter by Comment Counts
- Top 1% active actors?

In [5]:
N1 = int(cmt_qty_by_acc.shape[0]/100)
top1 = pd.Series(cmt_qty_by_acc.values[:N1])
top1.index = cmt_qty_by_acc.index[:N1]
print(top1.shape)

(952,)


In [6]:
top1.describe()

count    952.000000
mean       8.152311
std        3.822414
min        5.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       40.000000
dtype: float64

### Calculate Unique Post Count

In [7]:
post_qty_by_acc = [len(set(df.postId.values[df.profile_id==acc])) for acc in tqdm(top1.index)]

top1df = pd.DataFrame({
    'cmt_qty':top1.values,
    'post_qty': post_qty_by_acc
})
top1df.index = top1.index
print('\n', top1df.shape)
top1df['provocation_ratio'] = top1df.post_qty / top1df.cmt_qty
top1df.sort_values(['provocation_ratio','cmt_qty'], ascending=False,inplace=True)

100%|██████████| 952/952 [00:11<00:00, 82.33it/s]

 (952, 2)


In [8]:
top1df.to_csv('../results/20200413_20200420_top1_provocation.csv')

## Find Lex Found Sentences

In [10]:
df1 = df.loc[df.profile_id.isin(top1df.index),:]
df1['lex_count'] = [0 if na else 1 for na in df1.LexFound.isna()]

In [11]:
# groupby profileid
lex_qty_by_acc = df1.groupby('profile_id')['lex_count'].sum()
lex_qty_by_acc

profile_id
100000057365715    0
100000101995671    0
100000102677211    0
100000137724189    0
100000142373085    6
                  ..
1737084786         1
1775949670         1
1816629647         0
498314813612104    0
543905947          0
Name: lex_count, Length: 952, dtype: int64

In [12]:
top1df['lex_count'] = lex_qty_by_acc
top1df.head()

,cmt_qty,post_qty,provocation_ratio,lex_count
profile_id,,,,
100028019040288,10,10,1.0,0
100023359543448,9,9,1.0,0
100016344181312,9,9,1.0,5
100016930425001,9,9,1.0,3
100022162780898,8,8,1.0,0


In [13]:
top1df.sort_values(
    ['lex_count','provocation_ratio'],
    ascending=False)

,cmt_qty,post_qty,provocation_ratio,lex_count
profile_id,,,,
100009583100175,23,3,0.130435,19
100014938440875,33,1,0.030303,13
100008344413856,19,1,0.052632,10
100012465663554,19,1,0.052632,9
100044060053317,13,6,0.461538,6
...,...,...,...,...
100042149160289,18,1,0.055556,0
100038945359152,18,1,0.055556,0
100015035296352,22,1,0.045455,0


In [14]:
top1df['rank1'] = top1df.provocation_ratio * top1df.lex_count
top1df.sort_values(['rank1'], ascending=False).head(10)

,cmt_qty,post_qty,provocation_ratio,lex_count,rank1
profile_id,,,,,
100016344181312,9,9,1.000000,5,5.000000
100041595170810,8,8,1.000000,4,4.000000
100017790403344,11,10,0.909091,4,3.636364
100016930425001,9,9,1.000000,3,3.000000
100044060053317,13,6,0.461538,6,2.769231
100000142373085,12,5,0.416667,6,2.500000
100009583100175,23,3,0.130435,19,2.478261
100010831205744,10,8,0.800000,3,2.400000
100041802285557,10,6,0.600000,4,2.400000


In [15]:
top1df['rank2'] = top1df.post_qty * top1df.lex_count
top1df.sort_values(['rank2'], ascending=False).head(10)

,cmt_qty,post_qty,provocation_ratio,lex_count,rank1,rank2
profile_id,,,,,,
100009583100175,23,3,0.130435,19,2.478261,57
100016344181312,9,9,1.000000,5,5.000000,45
100017790403344,11,10,0.909091,4,3.636364,40
100044060053317,13,6,0.461538,6,2.769231,36
100013299252808,15,7,0.466667,5,2.333333,35
100041595170810,8,8,1.000000,4,4.000000,32
100000142373085,12,5,0.416667,6,2.500000,30
100016930425001,9,9,1.000000,3,3.000000,27
100049645520278,12,9,0.750000,3,2.250000,27


In [16]:
top1df['rank3'] = top1df.lex_count / top1df.cmt_qty
top1df.sort_values(['rank3'], ascending=False).head(10)

,cmt_qty,post_qty,provocation_ratio,lex_count,rank1,rank2,rank3
profile_id,,,,,,,
100029101801142,6,2,0.333333,5,1.666667,10,0.833333
100009583100175,23,3,0.130435,19,2.478261,57,0.826087
100022107270371,8,1,0.125000,6,0.750000,6,0.750000
100042465318376,6,1,0.166667,4,0.666667,4,0.666667
100016344181312,9,9,1.000000,5,5.000000,45,0.555556
100047181815568,11,1,0.090909,6,0.545455,6,0.545455
100008344413856,19,1,0.052632,10,0.526316,10,0.526316
100015101389061,6,4,0.666667,3,2.000000,12,0.500000
100047359334741,6,1,0.166667,3,0.500000,3,0.500000


In [17]:
top1df.sort_values(
    ['rank1','rank3','rank2'],
    ascending=False,
    inplace=True
)

In [18]:
top1df['rank4'] = top1df.provocation_ratio * top1df.rank3 #=top1df.rank1 / top1df.cmt_qty
top1df.sort_values('rank4', ascending=False).head(10)

,cmt_qty,post_qty,provocation_ratio,lex_count,rank1,rank2,rank3,rank4
profile_id,,,,,,,,
100016344181312,9,9,1.000000,5,5.000000,45,0.555556,0.555556
100041595170810,8,8,1.000000,4,4.000000,32,0.500000,0.500000
100015944012631,5,5,1.000000,2,2.000000,10,0.400000,0.400000
100016930425001,9,9,1.000000,3,3.000000,27,0.333333,0.333333
100015101389061,6,4,0.666667,3,2.000000,12,0.500000,0.333333
100017790403344,11,10,0.909091,4,3.636364,40,0.363636,0.330579
100046162694115,5,4,0.800000,2,1.600000,8,0.400000,0.320000
100010285562417,6,5,0.833333,2,1.666667,10,0.333333,0.277778
100029101801142,6,2,0.333333,5,1.666667,10,0.833333,0.277778


In [19]:
N = 40
d = [
    top1df.sort_values('rank1').tail(N).index,
    top1df.sort_values('rank2').tail(N).index,
    top1df.sort_values('rank3').tail(N).index,
    top1df.sort_values('rank4').tail(N).index
]
res = list(set.intersection(*map(set, d)))
print(len(res))

15


In [21]:
top1df.loc[res,:].to_csv('../results/20200413_20200420_Top40.csv')